In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [1]:
# ปรับแต่งค่าการทำงานของ Spark
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("KafkaSubscribe").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1000m").\
        config("spark.executor.cores", "2").\
        config("spark.cores.max", "6").\
        config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0").\
        getOrCreate()



Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-50dc416a-2c15-42a5-ba8a-9437ba459e73;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 623ms :: artifacts dl 16

In [2]:
rawMetadata_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "10.128.0.12:9092") \
  .option("subscribe", "quickstart-events") \
  .option("group.id", "Aekanun-Spark-App") \
  .load()

In [3]:
from pyspark.sql import functions as sparkf

In [4]:
rawMetadata_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [5]:
# เลือกเฉพาะคอลัมน์ที่ต้องการ
onlyMetadata_df = rawMetadata_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [6]:
from pyspark.sql.types import *
from pyspark.sql.functions import from_json, col

# Main schema
schema = StructType([
    StructField("locationId", StringType()),
    StructField("location", StringType()),
    StructField("parameter", StringType()),
    StructField("value", StringType()),
    StructField("date", StringType()),
    StructField("unit", StringType()),
    StructField("coordinates", StringType()),
    StructField("country", StringType()),
    StructField("city", StringType()),
    StructField("isMobile", StringType()),
    StructField("isAnalysis", StringType()),
    StructField("entity", StringType()),
    StructField("sensorType", StringType())
])

# Nested 'date' and 'coordinates' schema
date_schema = StructType([
    StructField("utc", StringType()),
    StructField("local", StringType())
])

coordinates_schema = StructType([
    StructField("latitude", DoubleType()),
    StructField("longitude", DoubleType())
])

# Parse the JSON string column and convert it to a struct.
parsedData_df = onlyMetadata_df.withColumn("data", from_json("value", schema))

In [7]:
parsedData_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- locationId: string (nullable = true)
 |    |-- location: string (nullable = true)
 |    |-- parameter: string (nullable = true)
 |    |-- value: string (nullable = true)
 |    |-- date: string (nullable = true)
 |    |-- unit: string (nullable = true)
 |    |-- coordinates: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- isMobile: string (nullable = true)
 |    |-- isAnalysis: string (nullable = true)
 |    |-- entity: string (nullable = true)
 |    |-- sensorType: string (nullable = true)



In [8]:
# Select only the nested fields
unNested_df = parsedData_df.select("key", "data.*")

In [9]:
unNested_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: string (nullable = true)
 |-- date: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- coordinates: string (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)



In [10]:
# Parse the JSON string column and convert it to a struct.
unNested_df.withColumn("data", from_json("value", schema))\
.withColumn("date", from_json("date", date_schema))\
.withColumn("coordinates", from_json("coordinates", coordinates_schema))\
.printSchema()

root
 |-- key: string (nullable = true)
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: string (nullable = true)
 |-- date: struct (nullable = true)
 |    |-- utc: string (nullable = true)
 |    |-- local: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- locationId: string (nullable = true)
 |    |-- location: string (nullable = true)
 |    |-- parameter: string (nullable = true)
 |    |-- value: string (nullable = true)
 |    |-- date: string (nullable = true)
 |    |-- unit: st

In [11]:
extractedDateLatLong_df = unNested_df.withColumn("data", from_json("value", schema))\
.withColumn("date", from_json("date", date_schema))\
.withColumn("coordinates", from_json("coordinates", coordinates_schema))\
.select('key',
 'locationId',
 'location',
 'parameter',
 'value',
 'date.*',
 'unit',
 'coordinates.*',
 'country',
 'city',
 'isMobile',
 'isAnalysis',
 'entity',
 'sensorType')

In [12]:
extractedDateLatLong_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: string (nullable = true)
 |-- utc: string (nullable = true)
 |-- local: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)



In [13]:
final_df = extractedDateLatLong_df\
.withColumn('sourceUnixSTP',sparkf.to_timestamp(sparkf.col('utc'), "yyyy-MM-dd'T'HH:mm:ssXXX"))

In [14]:
final_df.columns

['key',
 'locationId',
 'location',
 'parameter',
 'value',
 'utc',
 'local',
 'unit',
 'latitude',
 'longitude',
 'country',
 'city',
 'isMobile',
 'isAnalysis',
 'entity',
 'sensorType',
 'sourceUnixSTP']

In [15]:
final_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: string (nullable = true)
 |-- utc: string (nullable = true)
 |-- local: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)
 |-- sourceUnixSTP: timestamp (nullable = true)



In [16]:
#agg_df = final_df.groupBy('value').agg(sparkf.max(sparkf.col('value')))

In [17]:
from pyspark.sql import functions as F

In [ ]:
from pyspark.sql.functions import window

windowedCounts = final_df.groupBy(
    window(final_df.sourceUnixSTP, "3600 minutes", "3600 minutes"),
    final_df.value
).count()

orderwindowedCounts = windowedCounts.orderBy('window')

query = orderwindowedCounts.writeStream.outputMode("complete").format("console").start()
query.awaitTermination()


23/06/19 23:33:05 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-4f275f9e-68f9-44bd-aea9-1ac1da4b4bdd. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/06/19 23:33:06 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-66455927-e270-4018-9a90-02b4bc311cef-1209735755-driver-0-1, groupId=spark-kafka-source-66455927-e270-4018-9a90-02b4bc311cef-1209735755-driver-0] Error while fetching metadata with correlation id 2 : {quickstart-events=LEADER_NOT_AVAILABLE}
23/06/19 23:33:06 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-66455927-e270-4018-9a90-02b4bc311cef-1209735755-driver-0-1, groupId=spark-kafka-source-66455927-e270-4018-9a90-02b4bc311cef-1209735755-driver-0] Error while fetching metadata with correlation id 4 : {quic

-------------------------------------------
Batch: 0
-------------------------------------------
+------+-----+-----+
|window|value|count|
+------+-----+-----+
+------+-----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+-----+-----+
|              window|value|count|
+--------------------+-----+-----+
|[2022-12-31 12:00...| 36.0|    1|
+--------------------+-----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+-----+-----+
|              window|value|count|
+--------------------+-----+-----+
|[2022-12-31 12:00...| 31.0|    6|
|[2022-12-31 12:00...| 32.0|    2|
|[2022-12-31 12:00...| 35.0|    2|
|[2022-12-31 12:00...| 34.0|    2|
|[2022-12-31 12:00...| 37.0|    8|
|[2022-12-31 12:00...| 36.0|    8|
|[2022-12-31 12:00...| 33.0|    3|
|[2022-12-31 12:00...| 29.0|    8|
|[2022-12-31 12:00...| 30.0|    6|
|[2022-12-31 12:00...| 38.0|    8|
|[2023-01-03 00:00...| 35.0|    1|
|[2023-01-03 00:00...| 30.0|    1|
|[2023-01-03 00:00...| 33.0|    2|
|[2023-01-03 00:00...| 32.0|    8|
|[2023-01-03 00:00...| 31.0|    4|
|[2023-01-03 00:00...| 37.0|    6|
|[2023-01-03 00:00...| 34.0|    2|
|[2023-01-03 00:00...| 36.0|    3|
+--------------------+-----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+-----+-----+
|              window|value|count|
+--------------------+-----+-----+
|[2022-12-31 12:00...| 38.0|    8|
|[2022-12-31 12:00...| 29.0|    8|
|[2022-12-31 12:00...| 30.0|    6|
|[2022-12-31 12:00...| 33.0|    3|
|[2022-12-31 12:00...| 31.0|    6|
|[2022-12-31 12:00...| 32.0|    2|
|[2022-12-31 12:00...| 36.0|    8|
|[2022-12-31 12:00...| 35.0|    2|
|[2022-12-31 12:00...| 37.0|    8|
|[2022-12-31 12:00...| 34.0|    2|
|[2023-01-03 00:00...| 34.0|    2|
|[2023-01-03 00:00...| 28.0|    2|
|[2023-01-03 00:00...| 31.0|    4|
|[2023-01-03 00:00...| 32.0|    8|
|[2023-01-03 00:00...| 30.0|    8|
|[2023-01-03 00:00...| 29.0|   15|
|[2023-01-03 00:00...| 35.0|    1|
|[2023-01-03 00:00...| 33.0|    2|
|[2023-01-03 00:00...| 37.0|    6|
|[2023-01-03 00:00...| 36.0|    3|
+--------------------+-----+-----+
only showing top 20 rows



...

-------------------------------------------
Batch: 18
-------------------------------------------
+--------------------+-----+-----+
|              window|value|count|
+--------------------+-----+-----+
|[2022-12-31 12:00...| 31.0|    6|
|[2022-12-31 12:00...| 32.0|    2|
|[2022-12-31 12:00...| 35.0|    2|
|[2022-12-31 12:00...| 34.0|    2|
|[2022-12-31 12:00...| 37.0|    8|
|[2022-12-31 12:00...| 36.0|    8|
|[2022-12-31 12:00...| 38.0|    8|
|[2022-12-31 12:00...| 29.0|    8|
|[2022-12-31 12:00...| 30.0|    6|
|[2022-12-31 12:00...| 33.0|    3|
|[2023-01-03 00:00...| 35.0|    1|
|[2023-01-03 00:00...| 29.0|   15|
|[2023-01-03 00:00...| 30.0|    8|
|[2023-01-03 00:00...| 33.0|    2|
|[2023-01-03 00:00...| 32.0|    8|
|[2023-01-03 00:00...| 36.0|    3|
|[2023-01-03 00:00...| 31.0|    4|
|[2023-01-03 00:00...| 34.0|    2|
|[2023-01-03 00:00...| 28.0|    2|
|[2023-01-03 00:00...| 37.0|    6|
+--------------------+-----+-----+
only showing top 20 rows



-------------------------------------------
Batch: 19
-------------------------------------------
+--------------------+-----+-----+
|              window|value|count|
+--------------------+-----+-----+
|[2022-12-31 12:00...| 31.0|    6|
|[2022-12-31 12:00...| 32.0|    2|
|[2022-12-31 12:00...| 35.0|    2|
|[2022-12-31 12:00...| 34.0|    2|
|[2022-12-31 12:00...| 37.0|    8|
|[2022-12-31 12:00...| 36.0|    8|
|[2022-12-31 12:00...| 38.0|    8|
|[2022-12-31 12:00...| 29.0|    8|
|[2022-12-31 12:00...| 33.0|    3|
|[2022-12-31 12:00...| 30.0|    6|
|[2023-01-03 00:00...| 35.0|    1|
|[2023-01-03 00:00...| 29.0|   15|
|[2023-01-03 00:00...| 30.0|    8|
|[2023-01-03 00:00...| 33.0|    2|
|[2023-01-03 00:00...| 32.0|    8|
|[2023-01-03 00:00...| 31.0|    4|
|[2023-01-03 00:00...| 34.0|    2|
|[2023-01-03 00:00...| 28.0|    2|
|[2023-01-03 00:00...| 37.0|    6|
|[2023-01-03 00:00...| 36.0|    3|
+--------------------+-----+-----+
only showing top 20 rows



-------------------------------------------
Batch: 20
-------------------------------------------
+--------------------+-----+-----+
|              window|value|count|
+--------------------+-----+-----+
|[2022-12-31 12:00...| 31.0|    6|
|[2022-12-31 12:00...| 32.0|    2|
|[2022-12-31 12:00...| 35.0|    2|
|[2022-12-31 12:00...| 34.0|    2|
|[2022-12-31 12:00...| 37.0|    8|
|[2022-12-31 12:00...| 36.0|    8|
|[2022-12-31 12:00...| 30.0|    6|
|[2022-12-31 12:00...| 33.0|    3|
|[2022-12-31 12:00...| 38.0|    8|
|[2022-12-31 12:00...| 29.0|    8|
|[2023-01-03 00:00...| 35.0|    1|
|[2023-01-03 00:00...| 29.0|   15|
|[2023-01-03 00:00...| 30.0|    8|
|[2023-01-03 00:00...| 33.0|    2|
|[2023-01-03 00:00...| 32.0|    8|
|[2023-01-03 00:00...| 31.0|    4|
|[2023-01-03 00:00...| 34.0|    2|
|[2023-01-03 00:00...| 28.0|    2|
|[2023-01-03 00:00...| 37.0|    6|
|[2023-01-03 00:00...| 36.0|    3|
+--------------------+-----+-----+
only showing top 20 rows



-------------------------------------------
Batch: 21
-------------------------------------------
+--------------------+-----+-----+
|              window|value|count|
+--------------------+-----+-----+
|[2022-12-31 12:00...| 38.0|    8|
|[2022-12-31 12:00...| 29.0|    8|
|[2022-12-31 12:00...| 30.0|    6|
|[2022-12-31 12:00...| 33.0|    3|
|[2022-12-31 12:00...| 37.0|    8|
|[2022-12-31 12:00...| 35.0|    2|
|[2022-12-31 12:00...| 34.0|    2|
|[2022-12-31 12:00...| 36.0|    8|
|[2022-12-31 12:00...| 31.0|    6|
|[2022-12-31 12:00...| 32.0|    2|
|[2023-01-03 00:00...| 34.0|    2|
|[2023-01-03 00:00...| 28.0|    2|
|[2023-01-03 00:00...| 31.0|    4|
|[2023-01-03 00:00...| 37.0|    6|
|[2023-01-03 00:00...| 36.0|    3|
|[2023-01-03 00:00...| 32.0|    8|
|[2023-01-03 00:00...| 35.0|    1|
|[2023-01-03 00:00...| 29.0|   15|
|[2023-01-03 00:00...| 30.0|    8|
|[2023-01-03 00:00...| 33.0|    2|
+--------------------+-----+-----+
only showing top 20 rows



-------------------------------------------
Batch: 22
-------------------------------------------
+--------------------+-----+-----+
|              window|value|count|
+--------------------+-----+-----+
|[2022-12-31 12:00...| 38.0|    8|
|[2022-12-31 12:00...| 29.0|    8|
|[2022-12-31 12:00...| 30.0|    6|
|[2022-12-31 12:00...| 33.0|    3|
|[2022-12-31 12:00...| 37.0|    8|
|[2022-12-31 12:00...| 35.0|    2|
|[2022-12-31 12:00...| 34.0|    2|
|[2022-12-31 12:00...| 36.0|    8|
|[2022-12-31 12:00...| 31.0|    6|
|[2022-12-31 12:00...| 32.0|    2|
|[2023-01-03 00:00...| 34.0|    2|
|[2023-01-03 00:00...| 28.0|    2|
|[2023-01-03 00:00...| 31.0|    4|
|[2023-01-03 00:00...| 37.0|    6|
|[2023-01-03 00:00...| 36.0|    3|
|[2023-01-03 00:00...| 32.0|    8|
|[2023-01-03 00:00...| 35.0|    1|
|[2023-01-03 00:00...| 29.0|   15|
|[2023-01-03 00:00...| 30.0|    8|
|[2023-01-03 00:00...| 33.0|    2|
+--------------------+-----+-----+
only showing top 20 rows



In [ ]:
query.stop()